In [ ]:
%%sql

CREATE TABLE analytics.high_efficiency_category AS
SELECT
  card_tpbuz_nm_1,
  card_tpbuz_nm_2,
  SUM(amt) AS total_amt,
  SUM(cnt) AS total_cnt,
  ROUND(SUM(amt)::FLOAT / NULLIF(SUM(cnt), 0), 2) AS avg_amt_per_txn,
  RANK() OVER (ORDER BY ROUND(SUM(amt)::FLOAT / NULLIF(SUM(cnt), 0), 2) DESC) AS efficiency_rank
FROM raw_data.card_transaction
GROUP BY card_tpbuz_nm_1, card_tpbuz_nm_2
ORDER BY avg_amt_per_txn DESC;


In [ ]:
%%sql

CREATE TABLE analytics.efficient_category_gender_analysis AS
SELECT
  h.card_tpbuz_nm_1,
  h.card_tpbuz_nm_2,
  g.sex,
  g.total_amt,
  g.total_cnt,
  g.avg_amt_per_txn
FROM analytics.high_efficiency_category h
JOIN analytics.gender_industry_spending_stats g
  ON h.card_tpbuz_nm_1 = g.card_tpbuz_nm_1
 AND h.card_tpbuz_nm_2 = g.card_tpbuz_nm_2
WHERE h.efficiency_rank <= 10;


In [1]:
%%sql

CREATE TABLE analytics.efficient_category_monthly_trend AS
SELECT
  h.card_tpbuz_nm_1,
  h.card_tpbuz_nm_2,
  m.month,
  m.total_amt,
  m.total_cnt,
  m.avg_unit_price
FROM analytics.high_efficiency_category h
JOIN analytics.monthly_sales_by_industry m
  ON h.card_tpbuz_nm_1 = m.card_tpbuz_nm_1
 AND h.card_tpbuz_nm_2 = m.card_tpbuz_nm_2
WHERE h.efficiency_rank <= 10;


UsageError: Cell magic `%%sql` not found.


In [ ]:
%%sql

-- F → 여성
UPDATE analytics.efficient_category_gender_analysis
SET sex = '여성'
WHERE sex = 'F';

-- M → 남성
UPDATE analytics.efficient_category_gender_analysis
SET sex = '남성'
WHERE sex = 'M';